In [54]:
import pandas as pd

In [55]:
model_name = 'sd3'

In [56]:
mappings = []
mappings_dict = {}
with open('./order_raw.txt', 'r') as f:
    for line in f.readlines():
        temp = line.split('\t')

        try:
            mapping = {
                'model_name' : temp[0].strip().split('_')[0],
                'type_name' : temp[0].strip().split('_')[1],
                'img_id' : temp[1].strip().split('_')[1],
                'img_name' : temp[1].strip(),
                'qual_id' : temp[2].strip(),
            }

            mappings_dict[temp[2].strip()] = {
                'model_name' : temp[0].strip().split('_')[0],
                'type_name' : temp[0].strip().split('_')[1],
                'img_id' : temp[1].strip().split('_')[1],
                'img_name' : temp[1].strip(),
                'qual_id' : temp[2].strip(),
            }
        except:
            mapping = {
                'model_name' : temp[0].strip(),
                'img_id' : temp[1].strip().split('.')[0],
                'img_name' : temp[1].strip(),
                'qual_id' : temp[2].strip(),
                'type_name' : 'gt',
            }
            mappings_dict[temp[2].strip()] ={
                'model_name' : temp[0].strip(),
                'img_id' : temp[1].strip().split('.')[0],
                'img_name' : temp[1].strip(),
                'qual_id' : temp[2].strip(),
                'type_name' : 'gt',
            }
        mappings.append(mapping)

# print(mappings)

# for mapping in mappings:
#     if mapping['model_name'] == 'gt':
#         print(mapping)


In [57]:
import re

def extract_image_id(text):
    # 使用正则表达式匹配以 "IM_" 开头的字母和数字组合
    match = re.search(r'IM_[A-Za-z0-9]+', text)
    if match:
        return match.group(0)  # 返回匹配的字符串
    return None  # 如果没有匹配则返回 None

In [58]:

if model_name == 'dalle':
    bias = 0
else:
    bias = 55

if model_name == 'fluxp':
    lack = 2
else:
    lack = 0


df = pd.read_csv(f'./first/{model_name}.csv')

# best[img_id]['baseline'/'cc'/'gt']
# medium[img_id]['baseline'/'cc'/'gt']
# worst[img_id]['baseline'/'cc'/'gt']

# [img_id][type][rank]

result = {}

img_ids = []

ccwinsbaseline = {}
cclosebaseline = {}


ranks = []
total_subject = len(df[f'QID{2+bias}_1'])
# 42 questions in total
for qid in range(2,42 - lack):
    # df[f'QID{i}_1'][0]                        #  Question 
    # df[f'QID{i}_1'][1]                        #  ID
    # df[f'QID{i}_1'][2]                        # First subject
    # df[f'QID{i}_1'][len(df[f'QID{i}_1'])-1]   # Last subject
    qual_id = {}
    data_qual_id = {}
    for idx in ['1','2','3']:
        # if extract_image_id(df[f'QID{qid+bias}_{idx}'][0]) == 'IM_KuXdjFWTjoUp045':
            # continue
        # print(extract_image_id(df[f'QID{qid+bias}_{idx}'][0]))
        qual_id[idx] = extract_image_id(df[f'QID{qid+bias}_{idx}'][0])
        data_qual_id[idx] = mappings_dict[qual_id[idx]]
        # try:
            
        #     qual_id[idx] = extract_image_id(df[f'QID{qid+bias}_{idx}'][0])
        #     data_qual_id[idx] = mappings_dict[qual_id[idx]]
        # except:
        #     print(qid)
        #     print(df[f'QID{qid+bias}_{idx}'][0])

    # print(data_qual_id)

    img_id = data_qual_id['1']['img_id']
    img_ids.append(img_id)

    result[img_id] = {
        'img_id': img_id,
        'baseline': {
            '1': 0,
            '2': 0,
            '3': 0,
        },
        'cc': {
            '1': 0,
            '2': 0,
            '3': 0,
        },
        'gt': {
            '1': 0,
            '2': 0,
            '3': 0,
        },
    }
    ccwinsbaseline[img_id] = 0
    cclosebaseline[img_id] = 0



    for subject_id in range(2, total_subject):
        temp_rank = {}
        for idx in ['1','2','3']:
            # print(df[f'QID{qid}_{idx}'][subject_id])
            rank = df[f'QID{qid+bias}_{idx}'][subject_id]
            type_name = data_qual_id[idx]['type_name']
            temp_rank[type_name] = rank
            # print(img_id, type_name, rank)
            result[img_id][type_name][rank] += 1

        if int(temp_rank['cc']) < int(temp_rank['baseline']):
            ccwinsbaseline[img_id] += 1
        else:
            cclosebaseline[img_id] += 1



        temp_ranks = {
            'img_id' : img_id,
            'baseline': temp_rank['baseline'],
            'cc': temp_rank['cc'],
            'gt': temp_rank['gt'],
            'subject_id': subject_id,
            'qual_id' : qual_id
        }
        ranks.append(temp_ranks)






In [59]:
import numpy as np
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

def fm_all(data):
    # print(data)
    # data[i][j][k] 表示第i个题目中第k个被试对第j种方法的排名
    results = []

    data = np.array(data)

    # 合并所有题目的排名数据
    # 将数据从形状 (num_items, num_subjects, num_methods) 转换为 (num_subjects*num_items, num_methods)
    num_items = data.shape[0]
    num_methods = data.shape[2]
    num_subjects = data.shape[1]

    # 重新构造 data，以便于进行一次性检验
    combined_data = np.concatenate([data[i] for i in range(num_items)], axis=0)

    # 对所有题目进行综合Friedman检验
    stat, p = friedmanchisquare(*[combined_data[:, j] for j in range(num_methods)])

    # 显示结果
    print(f"综合Friedman test statistic = {stat}, p-value = {p}")
    results.append(f"综合Friedman test statistic = {stat}, p-value = {p}")
    
    if p < 0.05:
        nemenyi_result = sp.posthoc_nemenyi_friedman(combined_data)
        print("Nemenyi post hoc results:")
        print(nemenyi_result)
        results.append("Nemenyi post hoc results:")
        results.append(f"{nemenyi_result}")

    return results

def fm(data, img_ids):
    for idx, d in enumerate(data):
        print(d, img_ids[idx])
    # 示例数据：10个被试对每个题目的排名
    # data[i][j][k] 表示第i个题目中第k个被试对第j种方法的排名
    data = np.array(data)

    # 合并所有题目的排名数据
    # 将数据从形状 (num_items, num_subjects, num_methods) 转换为 (num_subjects*num_items, num_methods)
    num_items = data.shape[0]
    num_methods = data.shape[2]
    num_subjects = data.shape[1]

    # 重新构造 data，以便于进行一次性检验
    
    results = []
    for idx, seperate_data in enumerate(data):

        # 对所有题目进行综合Friedman检验
        stat, p = friedmanchisquare(*[seperate_data[:, j] for j in range(num_methods)])

        # 显示结果
        print(f"综合Friedman test statistic = {stat}, p-value = {p} , Image id: {img_ids[idx]}")
        results.append(f"综合Friedman test statistic = {stat}, p-value = {p} , Image id: {img_ids[idx]}")

        if p < 0.05:
            nemenyi_result = sp.posthoc_nemenyi_friedman(seperate_data)
            print("Nemenyi post hoc results:")
            print(nemenyi_result)

    return results


# # fm partial
# data = []
# temp = []
# img_id = ranks[0]['img_id']
# temp_img_ids = [img_id]

# for rank in ranks:
#     # print(rank)
#     if rank['img_id']!=img_id:
#         data.append(temp)
#         img_id = rank['img_id']
#         temp_img_ids.append(img_id)
#         temp = []
#     temp.append([int(rank['baseline']), int(rank['cc']), int(rank['gt'])])


# data.append(temp)
# fm(data, temp_img_ids)


#fm_all
data = []
baseline_score = 0
cc_score = 0
gt_score = 0
print(len(ranks))
for rank in ranks:
    data.append([int(rank['baseline']), int(rank['cc']), int(rank['gt'])])
    baseline_score += int(rank['baseline'])
    cc_score += int(rank['cc'])
    gt_score += int(rank['gt'])

print(f'Result of {model_name}')
print('Baseline: ', baseline_score / len(ranks))
print('CC: ', cc_score / len(ranks))
print('GT: ', gt_score / len(ranks))
data = [data]
results = fm_all(data)


with open(f'./first/result_{model_name}.txt' ,'w') as f:
    f.write(f'Result of {model_name} \n')
    f.write(f'Baseline: {baseline_score / len(ranks)} \n')
    f.write(f'CC: {cc_score / len(ranks)} \n')
    f.write(f'GT: {gt_score / len(ranks)} \n')
    for result in results:
        f.write(f'{result} \n')

f.close()



400
Result of sd3
Baseline:  2.4725
CC:  2.195
GT:  1.3325
综合Friedman test statistic = 282.7349999999997, p-value = 4.026009817064263e-62
Nemenyi post hoc results:
          0         1    2
0  1.000000  0.000256  0.0
1  0.000256  1.000000  0.0
2  0.000000  0.000000  1.0


In [60]:

for img_id in img_ids:
    # print(result[img_id])
    print(img_id, ccwinsbaseline[img_id], cclosebaseline[img_id])

total_wins = sum([ccwinsbaseline[img_id] for img_id in img_ids])
total_lose = sum([cclosebaseline[img_id] for img_id in img_ids])

print(total_wins, total_lose)


    # break

# print(df)

139 2 8
144 8 2
163 8 2
200 7 3
22 9 1
241 9 1
267 10 0
276 10 0
27 8 2
300 9 1
301 8 2
302 7 3
30 0 10
341 7 3
353 8 2
35 3 7
375 2 8
377 7 3
379 8 2
3 10 0
401 9 1
426 4 6
432 6 4
452 8 2
45 4 6
49 7 3
524 4 6
536 2 8
543 8 2
547 9 1
569 7 3
56 1 9
605 6 4
63 0 10
662 3 7
69 7 3
70 9 1
73 6 4
78 4 6
7 6 4
250 150


In [61]:
count_id = {rank['img_id'] : 0 for rank in ranks}
print(len(ranks))
for rank in ranks:
    count_id[rank['img_id']]+= 1

for key in count_id.keys():
    print(count_id[key])

400
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [62]:
with open(f'./win_{model_name}.txt', 'w') as f:
    #  baseline v.s. gt
    f.write('baseline v.s. gt\n')
    wins = 0
    for rank in ranks:
        if int(rank['baseline']) < int(rank['gt']):
            wins += 1
    f.write(f'{wins} {len(ranks)} \n')
    f.write(f'{wins/len(ranks)} \n')

    #  cc v.s. gt
    f.write('cc v.s. gt\n')
    wins = 0
    for rank in ranks:
        if int(rank['cc']) < int(rank['gt']):
            wins += 1
    f.write(f'{wins} {len(ranks)} \n')
    f.write(f'{wins/len(ranks)} \n')

    #  cc v.s. baseline
    f.write('cc v.s. baseline\n')
    wins = 0
    for rank in ranks:
        if int(rank['cc']) < int(rank['baseline']):
            wins += 1
    f.write(f'{wins} {len(ranks)} \n')
    f.write(f'{wins/len(ranks)} \n')



In [63]:
print(model_name)
temp_rank = []
for rank in ranks:
    temp_rank.append( { 'baseline': int(rank['baseline']), 'cc': int(rank['cc']), 'gt': int(rank['gt'])})


# print(temp_rank)
    
# Convert to a DataFrame
df = pd.DataFrame(temp_rank)

# Calculate mean and variance for 'cc', 'baseline', and 'gt'
mean_values = df[['cc', 'baseline', 'gt']].mean()
variance_values = df[['cc', 'baseline', 'gt']].var()

# Display results
print("Mean values:\n", mean_values)
print("\nVariance values:\n", variance_values)



sd3
Mean values:
 cc          2.1950
baseline    2.4725
gt          1.3325
dtype: float64

Variance values:
 cc          0.433058
baseline    0.495482
gt          0.367863
dtype: float64
